<a href="https://colab.research.google.com/github/Gilangp/Machine-learning/blob/main/week%207/LAT4_JS07_2341720042_Gilang_Purnomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 4**

In [ ]:
!pip install annoy
!pip install faiss-cpu
!pip install faiss-gpu
!pip install hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551812 sha256=8a693172278f01d14e065d8df0635eaf84562236e3c18fce6646a6b71838b30e
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528147 sha256=e036339a012c6c96d82d43c70a98d552a41813d2b6840909fdb9090e804c465c
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/c

In [ ]:
import numpy as np
import time
from annoy import AnnoyIndex
import faiss
import hnswlib

n_data = 1_000_000
dim = 5
X = np.random.random((n_data, dim)).astype(np.float32)

query = np.random.random((1, dim)).astype(np.float32)
k = 10

print("=== Annoy ===")
ann_index = AnnoyIndex(dim, 'euclidean')

start = time.time()
for i in range(n_data):
    ann_index.add_item(i, X[i])
ann_index.build(10)
build_time = time.time() - start

start = time.time()
neighbors = ann_index.get_nns_by_vector(query[0], k, include_distances=True)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", neighbors[0][:5], "...")

print("\n=== FAISS (IndexFlatL2) ===")
faiss_index = faiss.IndexFlatL2(dim)

start = time.time()
faiss_index.add(X)
build_time = time.time() - start

start = time.time()
distances, indices = faiss_index.search(query, k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", indices[0][:5], "...")

print("\n=== HNSW (hnswlib) ===")
hnsw_index = hnswlib.Index(space='l2', dim=dim)

start = time.time()
hnsw_index.init_index(max_elements=n_data, ef_construction=200, M=16)
hnsw_index.add_items(X)
build_time = time.time() - start

hnsw_index.set_ef(50)

start = time.time()
labels, distances = hnsw_index.knn_query(query, k=k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", labels[0][:5], "...")

=== Annoy ===
Build time: 26.78817081451416 detik
Query time: 0.0021276473999023438 detik
Neighbors: [164590, 682052, 385190, 704999, 316163] ...

=== FAISS (IndexFlatL2) ===
Build time: 0.015662193298339844 detik
Query time: 0.006487607955932617 detik
Neighbors: [164590 682052 385190 704999 316163] ...

=== HNSW (hnswlib) ===
Build time: 174.1032886505127 detik
Query time: 0.0003032684326171875 detik
Neighbors: [164590 682052 385190 704999 316163] ...
